<a href="https://colab.research.google.com/github/yuukun123/Python/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

# Tùy chỉnh để hiển thị biểu đồ đẹp hơn
sns.set(style="whitegrid")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/Final_Cleaned_Database.xlsx")
print(df.to_string())

In [13]:
import pandas as pd

# Đường dẫn file
file_path = "/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/DelawarePainDatabase_SecondaryDatabase_FaceGenCharacterization.xlsx"
xls = pd.ExcelFile(file_path)

In [25]:
def clean_expression_sheet(df, required_cols=["Expression"]):
    # Xóa cột không tên
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # Xóa dòng thiếu cột bắt buộc
    df = df.dropna(subset=required_cols)

    # Loại bỏ các biểu cảm chứa 'pms13'
    df = df[~df["Expression"].str.lower().str.contains("pms13")]

    # Các cột cần ép kiểu và xử lý NaN
    numeric_cols = ["ThreatRating", "ConfusionRating", "ThreatComp", "ConfusionComp"]

    for col in numeric_cols:
        if col in df.columns:
            # Thay "." và chuỗi rỗng thành NaN
            df[col] = df[col].replace([".", ""], np.nan)

            # Ép kiểu float, xử lý lỗi bằng NaN
            df[col] = pd.to_numeric(df[col], errors="coerce")

            # Điền NaN bằng giá trị trung bình của cột
            df[col] = df[col].fillna(df[col].mean())

    # Bỏ cảnh báo downcasting
    df = df.infer_objects(copy=False)

    return df


ExpressionAverages

In [ ]:
df_expr = pd.read_excel(xls, sheet_name="ExpressionAverages")
# Xem kiểu dữ liệu của từng cột
print(df_expr.dtypes)
df_expr = clean_expression_sheet(df_expr)

print(df_expr.to_string())

FirstPassNorming

In [34]:
df_slider_mo = pd.read_excel(xls, sheet_name="Sliders_FaceGenModeller")
df_slider_mo = df_slider_mo.drop(columns=['PMS13'])
df_slider_mo = df_slider_mo.loc[:, ~df_slider_mo.columns.str.contains("^Unnamed")]

print(df_slider_mo.to_string())

            Slider   AK5   AJ2   EF5   KM4  KM4b  PMS5  PMS7 PMS14  PMS2  PMS8
0            Anger  0.22     .  0.84     .     .  0.95  0.07  0.75  0.35  0.15
1          Disgust  0.48  0.21  0.87  0.77  0.77     .     1  0.39  0.75     1
2             Fear     .     1     .     .     .     .     .     .     .     .
3              Sad  0.83     1     1     1     1     1     1     1     1     1
4      SmileClosed     .     .  0.46     1     1   0.7     1     .     1     1
5        SmileOpen     .   0.8  0.13     .   0.5  0.14  0.23  0.13     .     .
6         Surprise     .     .     .     .     .     .     .     .   0.4   0.2
7        BlinkLeft  0.56  0.15     .     .     .   0.2     .     .   0.3  0.13
8       BlinkRIght  0.57  0.14     .     .     .  0.18     .     .   0.3  0.17
9     BrowDownLeft     1     .  0.42     .     .     .     1     .   0.4  0.43
10   BrowDownRight     1     .  0.42     .     .     .     1     .  0.54  0.44
11      BrowInLeft     .     .     .     .     .    

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [27]:
df_first = pd.read_excel(xls, sheet_name="Sliders_FaceGenModeller")
print(df_first.columns.tolist())


['Slider', 'AK5', 'AJ2', 'EF5', 'KM4', 'KM4b', 'PMS5', 'PMS7', 'PMS14', 'PMS2', 'PMS8', 'PMS13', 'Unnamed: 12', 'Unnamed: 13']


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
